In [3]:
import pandas as pd

# Read all sheets into a dictionary of DataFrames
all_data = pd.read_excel('CPIHistoricalForecast.xlsx', sheet_name=None)

In [26]:
import pandas as pd
import numpy as np

def clean_sheet(df, year):
    # Skip non-data sheets
    if year == 'Instructions':
        return None
    
    # Find header row containing "Consumer Price Index item"
    header_mask = df.iloc[:, 0].astype(str).str.contains('Consumer Price Index item', na=False)
    if not header_mask.any():
        return None
    
    header_row = header_mask.idxmax()
    
    # Set headers and remove rows above
    df.columns = df.iloc[header_row]
    df = df.iloc[header_row+1:]
    
    # First column is categories
    df = df.rename(columns={df.columns[0]: 'Category'})
    df = df[df['Category'].notna()]
    df = df[~df['Category'].str.contains('About this table|Source:|Contact:', na=False)]
    
    # Add year column
    df['Year'] = year
    
    return df

# Load and clean data
all_data = pd.read_excel('CPIHistoricalForecast.xlsx', sheet_name=None)
processed_data = []

for sheet_name, df in all_data.items():
    cleaned_df = clean_sheet(df, sheet_name)
    if cleaned_df is None:
        continue
    
    # Get all columns that contain month-year (e.g., 'Jul-24')
    month_cols = [col for col in cleaned_df.columns if isinstance(col, str) and '-' in col and len(col.split('-')) == 2]
    
    # Process each month column and its subsequent forecast columns
    for i, month_col in enumerate(month_cols):
        month_pos = cleaned_df.columns.get_loc(month_col)
        
        # The next three columns should be Lower, Mid, Upper
        # But we need to verify they exist first
        if month_pos + 3 >= len(cleaned_df.columns):
            continue
            
        # Extract the values
        for _, row in cleaned_df.iterrows():
            category = row['Category']
            year = row['Year']
            
            # Get the forecast values
            lower = row.iloc[month_pos + 1]
            mid = row.iloc[month_pos + 2]
            upper = row.iloc[month_pos + 3]
            
            # Only add if we have valid values
            if pd.notna(lower) and pd.notna(mid) and pd.notna(upper):
                processed_data.append({
                    'Category': category,
                    'Year': year,
                    'Month_Year': month_col,
                    'Lower': lower,
                    'Mid': mid,
                    'Upper': upper
                })

# Create final DataFrame
final_df = pd.DataFrame(processed_data)

# Convert to numeric
final_df['Lower'] = pd.to_numeric(final_df['Lower'], errors='coerce')
final_df['Mid'] = pd.to_numeric(final_df['Mid'], errors='coerce')
final_df['Upper'] = pd.to_numeric(final_df['Upper'], errors='coerce')

# Parse dates
final_df[['Month', 'Year_short']] = final_df['Month_Year'].str.split('-', expand=True)
final_df['Year_full'] = '20' + final_df['Year_short']  # Adjust if needed for pre-2000 dates
final_df['Date'] = pd.to_datetime(final_df['Month'] + ' ' + final_df['Year_full'])

# Final clean DataFrame
final_df = final_df[['Date', 'Year', 'Category', 'Lower', 'Mid', 'Upper']].sort_values(['Category', 'Date'])
final_df = final_df.reset_index(drop=True)

# Save cleaned data
final_df.to_csv('cleaned_cpi_forecasts.csv', index=False)

# Verify
print("First 5 rows:")
print(final_df.head())
print("\nUnique Categories:", final_df['Category'].unique())
print("\nDate Range:", final_df['Date'].min(), "to", final_df['Date'].max())

First 5 rows:
        Date  Year  Category  Lower  Mid  Upper
0 2002-07-01  2003  All food    4.1 -0.2    1.8
1 2002-08-01  2003  All food    4.0 -0.2    1.8
2 2002-09-01  2003  All food    3.8  0.0    1.8
3 2002-10-01  2003  All food    3.7  0.2    1.9
4 2002-11-01  2003  All food    3.6  0.3    1.9

Unique Categories: ['All food' 'Beef and veal' 'Cereals and bakery products' 'Dairy products'
 'Eggs' 'Fats and oils' 'Fish and seafood' 'Food at home'
 'Food away from home' 'Fresh fruits' 'Fresh fruits and vegetables'
 'Fresh vegetables' 'Fruits and vegetables' 'Meats'
 'Meats, poultry, and fish' 'Nonalcoholic beverages' 'Other foods'
 'Other meats' 'Pork' 'Poultry' 'Processed fruits and vegetables'
 'Sugar and sweets']

Date Range: 2002-07-01 00:00:00 to 2025-01-01 00:00:00


C:\Users\vivek\AppData\Local\Temp\ipykernel_39356\31722257.py:83: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  final_df['Date'] = pd.to_datetime(final_df['Month'] + ' ' + final_df['Year_full'])


In [33]:
# Save to your current working directory
import os

# Specify the full file path (not just directory)
output_path = "C:/Users/vivek/Downloads/cleaned_cpi_forecasts.xlsx"

# Save the file
final_df.to_excel(output_path, index=False)

print(f"File successfully saved to: {output_path}")

File successfully saved to: C:/Users/vivek/Downloads/cleaned_cpi_forecasts.xlsx


In [44]:


# Load the Excel file
file_path = "PPIHistoricalForecast.xlsx"
xls = pd.ExcelFile(file_path)

# List all sheet names
print("Sheet Names:", xls.sheet_names)

# Load one sheet to inspect structure (e.g., 2003)
df_sample = pd.read_excel(file_path, sheet_name="2003")
print("\nSample Data (First 5 Rows):")
print(df_sample.head(5))

Sheet Names: ['2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', 'Instructions']

Sample Data (First 5 Rows):
  2025 Producer Price Index forecasts Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0                                 NaN        NaN        NaN        NaN   
1           Producer Price Index item        NaN     Jul-02        NaN   
2                                 NaN      Lower        Mid      Upper   
3                   Farm-level cattle      -22.5       -4.1       18.8   
4                      Wholesale beef      -13.9        0.6       17.5   

  Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ...  \
0        NaN        NaN        NaN        NaN        NaN        NaN  ...   
1        NaN     Aug-02        NaN        NaN     Sep-02        NaN  ...   
2      Lower        Mid      Upper      Lower        Mid      Upper  ...   
3      -

In [56]:
import pandas as pd
from datetime import datetime

def clean_sheet(sheet_name):
    # Read raw data without headers
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Find header rows
    header_row = df[df[0].str.contains("Producer Price Index item", na=False)].index[0]
    
    # Get month-year pairs (e.g., Jul-02 becomes July 2002)
    months = df.iloc[header_row, 1:].dropna().values
    
    # Create clean column names
    columns = ['Category']
    for month in months:
        month_str, year_str = month.split('-')
        full_year = 2000 + int(year_str) if int(year_str) < 50 else 1900 + int(year_str)
        month_name = datetime.strptime(month_str, '%b').strftime('%B')
        columns.extend([
            f"{month_name}_{full_year}_Lower",
            f"{month_name}_{full_year}_Mid", 
            f"{month_name}_{full_year}_Upper"
        ])
    
    # Set data starting from header_row+2
    df = df.iloc[header_row+2:]
    df.columns = columns[:len(df.columns)]
    
    # Drop empty rows
    return df.dropna(subset=['Category'])

def reshape_data(df, sheet_year):
    # Melt to long format
    id_vars = ['Category']
    df_long = pd.melt(df, id_vars=id_vars, var_name='Month_Year_Type', value_name='Value')
    
    # Split combined column
    split = df_long['Month_Year_Type'].str.split('_', expand=True)
    df_long['Month'] = split[0]
    df_long['Year'] = split[1].astype(int)
    df_long['Type'] = split[2]
    
    # Create proper date (first day of each month)
    df_long['Date'] = pd.to_datetime(df_long['Year'].astype(str) + '-' + df_long['Month'] + '-01')
    
    # Pivot to get Lower/Mid/Upper columns
    df_final = df_long.pivot_table(
        index=['Date', 'Year', 'Category'],
        columns='Type',
        values='Value'
    ).reset_index()
    
    return df_final[['Date', 'Year', 'Category', 'Lower', 'Mid', 'Upper']]

# Process all sheets
all_data = []
for sheet in xls.sheet_names:
    if sheet.isdigit():
        try:
            cleaned = clean_sheet(sheet)
            reshaped = reshape_data(cleaned, int(sheet))
            all_data.append(reshaped)
        except Exception as e:
            print(f"Skipping sheet {sheet}: {str(e)}")

final_df = pd.concat(all_data).sort_values(['Date', 'Category'])

# Save to CSV
final_df.to_csv("PPI_Forecasts_Structured.csv", index=False)
print("Final structured data:")
print(final_df.head())

Final structured data:
Type       Date  Year             Category Lower  Mid Upper
0    2002-07-01  2002    Farm-level cattle -22.5 -4.1  18.8
1    2002-07-01  2002      Farm-level eggs -43.5 -4.2  62.4
2    2002-07-01  2002    Farm-level fruits -32.3 -3.8  37.7
3    2002-07-01  2002      Farm-level milk -25.6 -2.7  28.1
4    2002-07-01  2002  Farm-level soybeans -29.1 -2.4  36.6
